<a href="https://colab.research.google.com/github/kimhwijin/HandsOnMachineLearing/blob/main/KFood_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
import csv

In [2]:
keras.backend.clear_session()
REDUCTION_FILTERS = {
    'Inception-v4' : {
        'k' : 192,
        'l' : 224,
        'm' : 256,
        'n' : 384
    },
    'Inception-ResNet-v2' : {
        'k' : 256,
        'l' : 256,
        'm' : 384,
        'n' : 384
    }
}
WEIGHTS_TYPE = 'Inception-v4'